# Simulate data

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import os
import sys
module_path = os.path.abspath(os.path.join('/gpfs/home/nonnenma/projects/emulators/simulators/L96'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from L96_base import f1, f2, J1, J1_init, f1_juliadef, f2_juliadef

def predictor_corrector(fun, y0, times, alpha=0.5):

    y = np.zeros((len(times), *y0.shape))
    y[0] = y0
    for i in range(1,len(times)):        
        dt = times[i] - times[i-1]

        f0 = fun(times[i-1], y[i-1])
        f1 = fun(times[i],   y[i-1] + dt*f0)

        y[i] = y[i-1] + dt * (alpha*f0 + (1-alpha)*f1)
        
    return y

In [ ]:
F, h, b, c = 10, 1, 10, 10
K = 36
T, dt = 5, 0.001

X_init = F * (0.5 + np.random.randn(K) * 1.0)
dX_dt = np.empty(X_init.size, dtype=X_init.dtype)

def fun(t, x):
    return f1(x, F, dX_dt, K)

times = np.linspace(0, T, np.floor(T/dt)+1)
out = predictor_corrector(fun=fun, y0=X_init.copy(), times=times, alpha=0.5)
plt.figure(figsize=(8,4))
plt.imshow(out.T, aspect='auto')
plt.xlabel('time')
plt.ylabel('location')
plt.show()

# Learn local emulator

In [ ]:
import torch
import os
import sys
module_path = os.path.abspath(os.path.join('/gpfs/home/nonnenma/projects/seasonal_forecasting/code/weatherbench'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.pytorch.layers import setup_conv, ResNetBlock, PeriodicConv2D
from src.pytorch.util import init_torch_device

device = init_torch_device()
dtype = torch.float32
dtype_np = np.float32

class Dataset(torch.utils.data.IterableDataset):
    def __init__(self, data, offset=1, 
                 start=None, end=None, 
                 normalize=False, randomize_order=True):

        self.data = data.copy()

        self.offset = offset
        if start is None or end is None:
            start, end = 0,  self.data.shape[0]-self.offset
        assert end > start
        self.start, self.end = start, end

        self.normalize = normalize
        self.mean, self.std = 0., 1.
        if self.normalize:
            self.mean = self.data.mean(axis=0).reshape(1,-1)
            self.std = self.data.std(axis=0).reshape(1,-1)
            self.data = (self.data - self.mean) / self.std 

        self.randomize_order = randomize_order

    def __getitem__(self, index):
        """ Generate one batch of data """
        return np.atleast_2d(self.data[np.asarray(index),:])

    def __iter__(self):
        """ Return iterable over data in random order """
        iter_start, iter_end = self.divide_workers()
        if self.randomize_order:
            idx = torch.randperm(iter_end - iter_start, device='cpu') + iter_start
        else: 
            idx = torch.arange(iter_start, iter_end, requires_grad=False, device='cpu')

        X = self.data[idx,:].reshape(len(idx), 1, -1)
        y = self.data[idx+self.offset,:].reshape(len(idx), 1, -1)

        return zip(X, y)

    def __len__(self):
        return self.data.shape[0]

    def divide_workers(self):
        """ parallelized data loading via torch.util.data.Dataloader """
        if torch.utils.data.get_worker_info() is None:
            iter_start = torch.tensor(self.start, requires_grad=False, dtype=torch.int, device='cpu')
            iter_end = torch.tensor(self.end, requires_grad=False, dtype=torch.int, device='cpu')
        else: 
            raise NotImplementedError('had no need for parallelization yet')

        return iter_start, iter_end

out32 = out.astype(dtype=dtype_np)
    
dg_train = Dataset(data=out32, offset=1, normalize=True, start=0, end=np.floor(out.shape[0]*0.8))
dg_val   = Dataset(data=out32, offset=1, normalize=True, 
                   start=np.ceil(out.shape[0]*0.8), end=np.floor(out.shape[0]*0.9))
batch_size = 32

validation_loader = torch.utils.data.DataLoader(
    dg_val, batch_size=batch_size, drop_last=False, num_workers=0 
)
train_loader = torch.utils.data.DataLoader(
    dg_train, batch_size=batch_size, drop_last=True, num_workers=0
)

In [ ]:
class TinyNetwork(torch.nn.Module):
    
    def __init__(self, n_filters, n_channels_in = 1, n_channels_out = 1):

        super(TinyNetwork, self).__init__()
        n_in = n_channels_in
        self.layers3x3 = []
        for i in range(len(n_filters)):
            n_out = n_filters[i]
            layer = torch.nn.Conv1d(in_channels = n_in, 
                                    out_channels = n_out, 
                                    kernel_size = 3, 
                                    padding = (3-1)//2, 
                                    bias = True, 
                                    padding_mode = 'zeros')
            self.layers3x3.append(layer)
            n_in = n_out
        self.layers3x3 = torch.nn.ModuleList(self.layers3x3)

        self.final = torch.nn.Conv1d(in_channels=n_in,
                                     out_channels=n_channels_out,
                                     kernel_size= 1)
        self.nonlinearity = torch.nn.ReLU()
    
    def forward(self, x):
        for layer in self.layers3x3:
            x = self.nonlinearity(layer(x))
        return self.final(x)

model = TinyNetwork(n_filters = [32, 32])

model.forward(torch.as_tensor(np.random.normal(size=(10, 1, 36)), device='cpu', dtype=dtype)).shape

In [ ]:
from src.pytorch.train import train_model

loss_fun = torch.nn.functional.mse_loss

train_out = train_model(model, train_loader, validation_loader, device, model_forward=model.forward, loss_fun=loss_fun, 
            lr=0.001, lr_min=1e-5, lr_decay=0.2, weight_decay=0.,
            max_epochs=200, max_patience=10, max_lr_patience=10, eval_every=None,
            verbose=True, save_dir=None)

training_loss = train_out['training_loss'][-1]
validation_loss = train_out['validation_loss']


# Evaluate model fit

In [ ]:
def model_simulate(y0, T):
    x = np.empty((T+1, *y0.shape))
    x[0] = y0.copy()
    xx = torch.as_tensor(x[0], device='cpu', dtype=dtype).reshape(1,1,-1)
    for i in range(1,T+1):
        xx = model.forward(xx)
        x[i] = xx.detach().numpy().copy()
    return x

out_model = model_simulate(y0=X_init.copy(), T=len(times))

plt.figure(figsize=(8,10))
plt.subplot(2,1,1)
plt.imshow(out32.T, aspect='auto')
plt.xlabel('time')
plt.ylabel('location')
plt.title('numerical simulation')
plt.subplot(2,1,2)
plt.imshow(out_model.T, aspect='auto')
plt.xlabel('time')
plt.ylabel('location')
plt.title('model-reconstructed simulation')

plt.show()

In [ ]:
plt.figure(figsize=(8,9))
plt.subplot(2,1,1)
plt.imshow(out32[:100].T - X_init.reshape(-1,1), aspect='auto')
plt.xlabel('time')
plt.ylabel('location')
plt.title('numerical simulation, differences to yo')
plt.colorbar()
plt.subplot(2,1,2)
plt.imshow(out_model[:100].T - X_init.reshape(-1,1), aspect='auto')
plt.xlabel('time')
plt.ylabel('location')
plt.title('model-reconstructed simulation, differences to yo')
plt.colorbar()

plt.show()

In [ ]:
plt.figure(figsize=(8,12))
for i in range(3):
    
    t0 = int(np.floor((i+1)*(len(times)-1)*0.25))
    T_ = 200
    y0 = out32[t0]

    out_model = model_simulate(y0=y0.copy(), T=T_)

    cmax = np.maximum((out32[t0:t0+T_].T - y0.reshape(-1,1)).max(),
                      (out_model.T - y0.reshape(-1,1)).max())
    cmin = np.minimum((out32[t0:t0+T_].T - y0.reshape(-1,1)).min(),
                      (out_model.T - y0.reshape(-1,1)).min())
    
    plt.subplot(3,2,2*i+1)
    plt.imshow(out32[t0:t0+T_].T - y0.reshape(-1,1), aspect='auto', vmax=cmax, vmin=cmin)
    
    
    if i == 2:
        plt.xlabel('time')
    plt.ylabel(f"T' = {(i+1)*25/100}*T")
    plt.title('numerical simulation')
    plt.colorbar()
    plt.subplot(3,2,2*i+2)
    plt.imshow(out_model.T - y0.reshape(-1,1), aspect='auto', vmax=cmax, vmin=cmin)
    if i == 2:
        plt.xlabel('time')
    plt.ylabel('location')
    plt.colorbar()
    plt.title('model-reconstructed simulation')

plt.show()